### **Modelo de Optimización Logística con Peajes y Restricciones de Peso – Caso 3**

#### **Ampliación del Modelo**
El Caso 3 representa la evolución del sistema logístico implementado en el Caso 2, integrando restricciones que reflejan condiciones reales del transporte terrestre en Colombia. Este modelo extiende la formulación MILP con:

1. Restricciones de peso por municipio: Límite máximo de carga permitida al ingresar a ciertos destinos.
2. Peajes diferenciados por tramo: Costos variables en función del peso transportado.
3. Cálculo detallado de los peajes pagados en cada arco recorrido.


#### **Componentes Extendidos del Modelo**

Parámetros Adicionales:
- MaxWeight: límite de ingreso (toneladas) por municipio.
- TollBase y TollVar: tarifa base y variable por tonelada asociada a cada arco.

Variables Nuevas:
- w[v, d]: peso transportado por el vehículo v al llegar al destino d.
- toll_cost[v, (i,j)]: costo total del peaje pagado por el vehículo v en el tramo (i,j).



#### **Restricciones Adicionales Incorporadas**

1. Restricción de Ingreso por Peso  
   Limita la carga que puede ingresar a municipios con restricciones:
   - $w[v,d] ≤ MaxWeight[d]$
   
2. Cálculo del Peaje por Arco Recorrido  
Para cada tramo `(i,j)`:
  - $toll\_cost[v,(i,j)]= TollBase[i,j]*x[v(i,j)]+TollVar[i,j]*ux[v,(i,j)]$

3. Inclusión del Peaje en la Función Objetivo  
Se suma el costo total de peajes pagados por cada vehículo:

- $total\_cost+= sum(toll_cost[v,(i,j)] ∀ A)$


In [1]:
import sys
!pip install ace-tools
if "google.colab" in sys.modules:

    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"

    import helper

    # Instalar las dependencias necesarias
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()

--2025-05-28 13:23:04--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py.1’

helper.py.1         100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-05-28 13:23:04 (48.8 MB/s) - ‘helper.py.1’ saved [7171/7171]

IDAES found! No need to install.
Installing glpk via apt-get...
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfuly installed
cbc was successfuly installed
clp was successfuly installed
bonmin was successfuly installed
couenne was successfuly installed
ipopt_l1 was successfuly installed
 


In [3]:
from google.colab import files
import io
import pandas as pd

print("Selecciona depots.csv, stations.csv, clients.csv, vehicles.csv y tolls.csv")
uploaded = files.upload()

dfs = {}
for key, val in uploaded.items():
    kl = key.lower()
    if 'depot'   in kl:
        dfs['depots']   = pd.read_csv(io.BytesIO(val))
    elif 'station' in kl:
        dfs['stations'] = pd.read_csv(io.BytesIO(val))
    elif 'client'  in kl:
        dfs['clients']  = pd.read_csv(io.BytesIO(val))
    elif 'vehicle' in kl:
        dfs['vehicles'] = pd.read_csv(io.BytesIO(val))
    elif 'tolls' in kl:
        dfs['tolls'] = pd.read_csv(io.BytesIO(val))
    else:
        print("Archivo no reconocido, ignorando:", key)

depots_df   = dfs['depots']
stations_df = dfs['stations']
clients_df  = dfs['clients']
vehicles_df = dfs['vehicles']
tolls_df = dfs['tolls']

Selecciona depots.csv, stations.csv, clients.csv, vehicles.csv y tolls.csv


Saving clients.csv to clients (2).csv
Saving depots.csv to depots (2).csv
Saving stations.csv to stations (2).csv
Saving tolls.csv to tolls (2).csv
Saving vehicles.csv to vehicles (2).csv


In [9]:
from pyomo.environ import *

def build_logistico_model(data):
    model = ConcreteModel()

    # --- Conjuntos ---
    model.V = Set(initialize=data['vehicles'])
    model.N = Set(initialize=data['nodes'])
    model.A = Set(initialize=data['arcs'], dimen=2)
    model.P = Set(initialize=data['ports'])
    model.E = Set(initialize=data['stations'])
    model.D = Set(initialize=data['destinations'])

    # --- Parámetros ---
    model.d  = Param(model.A, initialize=data['distances'])
    model.Cm = Param(initialize=data['Cm'])
    model.Ft = Param(initialize=data['Ft'])
    model.e  = Param(model.V, initialize=data['consumption'])
    model.a  = Param(model.V, initialize=data['autonomy'])
    model.Q  = Param(model.V, initialize=data['capacity'])
    model.q  = Param(model.D, initialize=data['demand'])
    model.W  = Param(model.D, initialize=data['weight_limit'])
    model.p  = Param(model.E, initialize=data['fuel_price'])
    model.Tb = Param(model.A, initialize=data['toll_base'], default=0)
    model.Tv = Param(model.A, initialize=data['toll_var'], default=0)
    model.M  = Param(initialize=data['BigM'])

    # --- Variables ---
    model.x         = Var(model.V, model.A, domain=Binary)
    model.b         = Var(model.V, model.N, domain=NonNegativeReals)
    model.y         = Var(model.V, model.E, domain=NonNegativeReals)
    model.delivered = Var(model.V, model.D, domain=NonNegativeReals)
    model.ux        = Var(model.V, model.A, domain=NonNegativeReals)
    model.ut = Var(model.V, model.N, domain=Integers,  bounds =(1 , len(model.N)-1) )
    model.w = Var(model.V, model.D, domain=NonNegativeReals)  # Peso del vehículo al llegar a un municipio
    model.toll_cost = Var(model.V, model.A, domain=NonNegativeReals)


    # --- Función Objetivo ---
    def obj_rule(m):
      return sum(
          m.d[i, j] * (m.Cm + m.Ft) * m.x[v, (i, j)] +
          m.d[i, j] * m.e[v] * (m.p[j] if j in m.E else 0) * m.x[v, (i, j)]
          for v in m.V for (i, j) in m.A
      ) + sum(m.toll_cost[v, (i, j)] for v in m.V for (i, j) in m.A)

    model.obj = Objective(rule=obj_rule, sense=minimize)


    # --- Restricciones de flujo ---
    model.depart    = Constraint(model.V, rule=lambda m,v:
                      sum(m.x[v,(p,j)] for p in m.P for j in m.N if p!=j) <= 1)

    model.flow      = Constraint(model.V, model.N, rule=lambda m,v,j:
                      (sum(m.x[v,(i,j)] for i in m.N if i!=j)
                     == sum(m.x[v,(j,k)] for k in m.N if j!=k))
                      if j not in m.P else Constraint.Skip)

    #model.no_return = Constraint(model.V, model.P, rule=lambda m,v,p:
    #                  sum(m.x[v,(j,p)] for j in m.N if (j,p) in m.A) == 0)

    n=len(model.N)-1
    def mtz_rule ( model ,k, i,j ):
      #i,j = a
      if i != j and i != 1 and j != 1:
        return model.ut[k, i] - model.ut[k,j] + n*model.x[k, (i ,j)] <= n - 1
      return Constraint.Skip

    def peso_en_municipio(m, v, d):
      return m.w[v, d] == m.delivered[v, d]
    model.calc_peso = Constraint(model.V, model.D, rule=peso_en_municipio)

    def limite_peso(m, v, d):
      return m.w[v, d] <= m.W[d]
    model.restriccion_peso = Constraint(model.V, model.D, rule=limite_peso)

    model.mtz_rules = Constraint(model.V, model.A, rule =  mtz_rule)

    def costo_peaje(m, v, i, j):
      return m.toll_cost[v, (i, j)] == m.Tb[i, j] * m.x[v, (i, j)] + m.Tv[i, j] * m.ux[v, (i, j)]
    model.calcular_peaje = Constraint(model.V, model.A, rule=costo_peaje)


    # --- Demanda y capacidad ---
    model.cap    = Constraint(model.V, rule=lambda m,v:
                      sum(m.delivered[v,j] for j in m.D) <= m.Q[v])
    model.demand = Constraint(model.D, rule=lambda m,j:
                      sum(m.delivered[v,j] for v in m.V) == m.q[j])

    # --- Solo entrega si visita ---
    model.link_delivery = Constraint(model.V, model.D, rule=lambda m,v,j:
                              m.delivered[v,j]
                            <= m.Q[v] * sum(m.x[v,(i,j)] for i in m.N if (i,j) in m.A))
#
    # --- Combustible y recarga ---
    model.init_fuel   = Constraint(model.V, rule=lambda m,v:
                              m.b[v,list(m.P)[0]] == m.a[v] * m.e[v])
    model.fuel_update = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                              m.b[v,j] <= m.b[v,i]
                                         - m.d[i,j]*m.e[v]
                                         + (m.y[v,j] if j in m.E else 0)
                                         + m.M*(1-m.x[v,(i,j)]))
    model.fuel_suff   = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                              m.b[v,i] >= m.d[i,j]*m.e[v]
                                         - m.M*(1-m.x[v,(i,j)]))
    model.recharge    = Constraint(model.V, model.E, rule=lambda m,v,j:
                              m.y[v,j] <= m.M * sum(m.x[v,(i,j)] for i in m.N if (i,j) in m.A))

    # --- Linealización de entrega ---
    model.lin1 = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                             m.ux[v,(i,j)] <= m.Q[v] * m.x[v,(i,j)])
    model.lin2 = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                             m.ux[v,(i,j)] <= (m.delivered[v,j] if j in m.D else 0))
    model.lin3 = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                             m.ux[v,(i,j)] >= (m.delivered[v,j] if j in m.D else 0)
                                               - m.Q[v]*(1-m.x[v,(i,j)]))

    return model


In [10]:
import pandas as pd
import math
from pyomo.environ import SolverFactory
#from google.colab import drive

# Coordenadas de municipios
city_coords = {
    'Bogotá':       (4.60971,  -74.08175),
    'Medellín':     (6.25184,  -75.56359),
    'Cali':         (3.43722,  -76.52250),
    'Cartagena':    (10.39972, -75.51444),
    'Cúcuta':       (7.89391,  -72.50782),
    'Bucaramanga':  (7.12539,  -73.11980),
    'Pereira':      (4.81333,  -75.69611),
    'Santa Marta':  (11.24079, -74.19904),
    'Ibagué':       (4.43889,  -75.23222),
    'Manizales':    (5.06889,  -75.51738),
    'Neiva':        (2.92730,  -75.28189),
    'Barranquilla': (10.96854, -74.78132),
    'Villavicencio':(4.14200,  -73.62664),
    'Armenia':      (4.53389,  -75.68111),
}

# Mapear lat/lon a clients_df
clients_df['Latitude']  = clients_df['City/Municipality'].map(lambda c: city_coords[c][0])
clients_df['Longitude'] = clients_df['City/Municipality'].map(lambda c: city_coords[c][1])

# Construir diccionario de ubicaciones
locations = {}
for _, r in depots_df.iterrows():
    locations[r['DepotID']] = (r['Latitude'], r['Longitude'])
for _, r in stations_df.iterrows():
    locations[r['EstationID']] = (r['Latitude'], r['Longitude'])
for _, r in clients_df.iterrows():
    locations[r['LocationID']] = (r['Latitude'], r['Longitude'])

# Definir nodos y arcos
nodes = list(locations.keys())
arcs = [(int(i), int(j)) for i in nodes for j in nodes if i != j]

# Calcular distancias Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))

distances = {(i, j): haversine(*locations[i], *locations[j]) for (i, j) in arcs}

# Preparar data para el modelo
data = {
    'vehicles':     vehicles_df['VehicleID'].tolist(),
    'nodes':        nodes,
    'arcs':         arcs,
    'ports':        depots_df['DepotID'].tolist(),
    'stations':     stations_df['EstationID'].tolist(),
    'destinations': clients_df['LocationID'].tolist(),
    'distances':    distances,
    'Cm':           700,
    'Ft':           5000,
    'consumption':  {vid: 0.2 for vid in vehicles_df['VehicleID']},
    'autonomy':     dict(zip(vehicles_df['VehicleID'], vehicles_df['Range'])),
    'capacity':     dict(zip(vehicles_df['VehicleID'], vehicles_df['Capacity'])),
    'demand':       dict(zip(clients_df['LocationID'], clients_df['Demand'])),
    'weight_limit': {did: float('inf') for did in clients_df['LocationID']},
    'fuel_price':   dict(zip(stations_df['EstationID'], stations_df['FuelCost'])),
    'toll_base':    {(i,j): 0 for (i,j) in arcs},
    'toll_var':     {(i,j): 0 for (i,j) in arcs},
    'BigM':         1e5
}
data['weight_limit'] = dict(zip(clients_df['LocationID'], clients_df['MaxWeight'].fillna(1e6)))
toll_base_dict = dict(zip(tolls_df['ClientID'], tolls_df['BaseRate'].fillna(0)))
toll_var_dict  = dict(zip(tolls_df['ClientID'], tolls_df['RatePerTon'].fillna(0)))

data['toll_base'] = {(i, j): toll_base_dict.get(j, 0) for (i, j) in arcs}
data['toll_var']  = {(i, j): toll_var_dict.get(j, 0) for (i, j) in arcs}


# Resolver el modelo
model = build_logistico_model(data)
solver = SolverFactory('appsi_highs')
solver.options['parallel'] = 'on'
solver.options['time_limit'] = 20 * 60
solver.options['presolve'] = 'on'
solver.solve(model, tee=True)

# Comprobar que demandas están satisfechas
print("Demandas vs. entregas:")
for j in data['destinations']:
    delivered = sum(model.delivered[v,j].value for v in data['vehicles'])
    print(f"  Nodo {j}: demanda={data['demand'][j]}, entregado={delivered}")


Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
MIP  has 9092 rows; 4194 cols; 27378 nonzeros; 1344 integer variables (1260 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+05]
  Cost   [1e+00, 8e+06]
  Bound  [1e+00, 1e+01]
  RHS    [1e+00, 1e+06]
Presolving model
7403 rows, 2758 cols, 23496 nonzeros  0s
7319 rows, 2758 cols, 23412 nonzeros  0s

Solving MIP model with:
   7319 rows
   2758 cols (1259 binary, 84 integer, 0 implied int., 1415 continuous)
   23412 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       

     37498    1727      4527   3.31%   16339260.5739   33490161.49137    51.21%     3571    202   9741     1223k  1200.0s
     37498    1727      4527   3.31%   16339260.5739   33490161.49137    51.21%     3571    202   9741     1223k  1200.0s

Solving report
  Status            Time limit reached
  Primal bound      33490161.4914
  Dual bound        16339260.5739
  Gap               51.21% (tolerance: 0.01%)
  P-D integral      675.938656188
  Solution status   feasible
                    33490161.4914 (objective)
                    0 (bound viol.)
                    3.07842640268e-12 (int. viol.)
                    0 (row viol.)
  Timing            1200.01 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 7
  Nodes             37498
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     1223964 (total)
                    265332 (strong br.)
                    72995 (separation)
   

In [11]:
for k in model.V:
  for i in model.A:
    if value(model.x[k,i])>=0.5:
      print(i, "es un nodo elegido para el vehiculo", k)

(1, 9) es un nodo elegido para el vehiculo 1
(2, 15) es un nodo elegido para el vehiculo 1
(3, 13) es un nodo elegido para el vehiculo 1
(5, 14) es un nodo elegido para el vehiculo 1
(9, 5) es un nodo elegido para el vehiculo 1
(13, 1) es un nodo elegido para el vehiculo 1
(14, 2) es un nodo elegido para el vehiculo 1
(15, 3) es un nodo elegido para el vehiculo 1
(1, 15) es un nodo elegido para el vehiculo 2
(4, 13) es un nodo elegido para el vehiculo 2
(8, 11) es un nodo elegido para el vehiculo 2
(10, 12) es un nodo elegido para el vehiculo 2
(11, 10) es un nodo elegido para el vehiculo 2
(12, 4) es un nodo elegido para el vehiculo 2
(13, 1) es un nodo elegido para el vehiculo 2
(15, 8) es un nodo elegido para el vehiculo 2
(1, 7) es un nodo elegido para el vehiculo 4
(6, 13) es un nodo elegido para el vehiculo 4
(7, 6) es un nodo elegido para el vehiculo 4
(13, 1) es un nodo elegido para el vehiculo 4


In [12]:
import folium
from folium import PolyLine, Marker, CircleMarker, Icon
from IPython.display import display

# 0) Normalizamos los arcos a tuplas de int
arcs_int = [(int(i), int(j)) for (i,j) in data['arcs']]

# 1) Creamos un mapa único, centrado en todas las ubicaciones
center = [
    sum(lat for lat,lon in locations.values()) / len(locations),
    sum(lon for lat,lon in locations.values()) / len(locations)
]
m_all = folium.Map(location=center, zoom_start=6)

# Colores distintos para cada vehículo
colors = ['blue','green','purple','orange','darkred','cadetblue','darkgreen']
# Si tienes más de len(colors) vehículos, los colores se irán repitiendo.

for idx, v in enumerate(data['vehicles']):
    color = colors[idx % len(colors)]
    usados = [(i,j) for (i,j) in arcs_int if model.x[v,(i,j)].value > 0.5]
    print(f"Vehículo {v} tramos activos:", usados)

    # 2) Dibujar cada tramo con el color asignado
    for (i,j) in usados:
        PolyLine(
            [locations[i], locations[j]],
            color=color, weight=3, opacity=0.8,
            popup=f'V{v}: {i}→{j}'
        ).add_to(m_all)

    # 3) Marcar nodos visitados: combustible y recargas
    nodos = set([i for i,j in usados] + [j for i,j in usados])
    for n in nodos:
        lat, lon = locations[n]
        # marcador nivel de combustible
        try:
            combustible = f"Fuel {model.b[v,n].value:.1f} gal"
        except :
            combustible = f"Fuel ignorado"
        CircleMarker(
            (lat, lon), radius=5,
            color=color, fill=True, fill_opacity=0.7,
            popup=f'V{v} Nodo {n}\n {combustible}'
        ).add_to(m_all)
        # marcador de recarga

        try:
            recarga = f"Recarga {model.y[v,n].value:.1f} gal"
            y=model.y[v,n].value
        except:
            recarga = f"Recarga ignorada"
            y=0

        if n in data['stations'] and y:
            Marker(
                (lat, lon),
                icon=Icon(color='lightblue', icon='tint'),
                popup=f'V{v}\n {recarga}'
            ).add_to(m_all)

    # 4) Dibujar peajes en esos tramos
    for (i,j) in usados:
        tb, tv = data['toll_base'][(i,j)], data['toll_var'][(i,j)]
        if tb > 0 or tv > 0:
            PolyLine(
                [locations[i], locations[j]],
                color='red', weight=2, dash_array='5,5',
                popup=f'Peaje {i}→{j}\nBase: {tb} COP\nVar: {tv} COP/ton'
            ).add_to(m_all)

# 5) Finalmente, mostramos el mapa completo
display(m_all)


Vehículo 1 tramos activos: [(1, 9), (2, 15), (3, 13), (5, 14), (9, 5), (13, 1), (14, 2), (15, 3)]
Vehículo 2 tramos activos: [(1, 15), (4, 13), (8, 11), (10, 12), (11, 10), (12, 4), (13, 1), (15, 8)]
Vehículo 3 tramos activos: []
Vehículo 4 tramos activos: [(1, 7), (6, 13), (7, 6), (13, 1)]
Vehículo 5 tramos activos: []
Vehículo 6 tramos activos: []
